In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, reset_output
from bokeh.models import HoverTool
from bokeh.models.annotations import Label, LabelSet
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.models import CategoricalColorMapper, ColorBar
from bokeh.transform import transform
from bokeh.models import LinearInterpolator
from bokeh.palettes import Category20b
import ipywidgets 
from bokeh.io import push_notebook
from bokeh.models import ColumnDataSource, Select
from bokeh.io import output_file, show
from bokeh.models import Dropdown, Slider
from bokeh.layouts import column, row
from bokeh.models.callbacks import CustomJS
from bokeh.io import curdoc



In [2]:
test_data = pd.read_csv("metro_data.csv", encoding = "latin1", sep = ";")
test_data.head()

,Timestamp,StimuliName,FixationIndex,FixationDuration,MappedFixationPointX,MappedFixationPointY,user,description
0,2586,01_Antwerpen_S1.jpg,9,250,1151,458,p1,color
1,2836,01_Antwerpen_S1.jpg,10,150,1371,316,p1,color
2,2986,01_Antwerpen_S1.jpg,11,283,1342,287,p1,color
3,3269,01_Antwerpen_S1.jpg,12,433,762,303,p1,color
4,3702,01_Antwerpen_S1.jpg,13,183,624,297,p1,color


In [11]:
# use only the 01_antwerpen_01 map


KeyError: False

In [4]:
source = ColumnDataSource(data=dict(Timestamp=[], StimuliName=[], FixationIndex=[], FixationDuration=[], MappedFixationPointX = [], MappedFixationPointY = [], user = []))



# add colours to the scatter points
color_mapper = CategoricalColorMapper ( 
    factors = list(test_data.user.unique()),
    palette = Category20b[20]
)




# set size of the dots in the future scatter plot
size_mapper = LinearInterpolator (
    x = [test_data.FixationDuration.min(), test_data.FixationDuration.max()],
    y = [10, 40]
)





C:\ProgramData\Anaconda3\lib\site-packages\bokeh\models\mappers.py:147: UserWarning: Palette length does not match number of factors. ['p28', 'p29', 'p3', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p4', 'p40', 'p5', 'p6', 'p7', 'p8', 'p9'] will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))


In [5]:
users = test_data['user'].unique().tolist()
users = sorted(users, key=lambda x: int("".join([i for i in x if i.isdigit()])))

maps = test_data['StimuliName'].unique().tolist()
maps.sort()

# dropdown menu to select a stimuli map
select_map = Select (
    title = "Select map:", 
    value = "01_Antwerpen_S1.jpg", 
    options = maps
)


# dropdown menu to select a user
select_user = Select (
    title = "Select user:", 
    value = "All",
    options = users
)


# slider for timestamp
slider_timestamp = Slider(
    start = test_data.Timestamp.min(), 
    end = test_data.Timestamp.max(), 
    step = 1,
    value = test_data.Timestamp.min(),
    title = 'Slide for time forward'
)


In [6]:
# function select the needed gprah according to the dropdown values

def select_graphs():
    stimulimap_val = select_map.value
    userselect_val = select_user.value.strip()
    
    selected = test_data.copy()
   
    if (stimulimap_val != "All"):
        selected = selected[selected['StimuliName'].str.contains(stimulimap_val)==True]
    if (userselect_val != "All"):
        selected = selected[selected['user'].str.contains(userselect_val)==True]
    
    return selected

# update the plot

def update():
    df = select_graphs()
    
    source.data = dict(
        Timestamp =  df['Timestamp'],
        StimuliName =  df['StimuliName'],
        FixationIndex =  df['FixationIndex'],
        FixationDuration =  df['FixationDuration'],
        MappedFixationPointX =  df['MappedFixationPointX'],
        MappedFixationPointY =  df['MappedFixationPointY'],
        user =  df['user']
        )

In [7]:
controls = [select_map, select_user, slider_timestamp]

for control in controls :
    control.on_change('value', lambda attr, old, new: update())

In [8]:
# scatter plot //  gaze plot
TOOLTIPS=[
    ("Time", "@Timestamp"),
    ("User", "@user"),
    ("X-coordinate", "@MappedFixationPointX"),
    ("Y-coordinate", "@MappedFixationPointY"),
]
gaze_plot = figure (plot_width = 1000, plot_height = 650, title = 'test gaze plot', tooltips = TOOLTIPS, x_axis_label = 'x', y_axis_label = 'y')
gaze_plot.circle (x = 'MappedFixationPointX',
                  y = 'MappedFixationPointY',
                  source = source,
                  color = {'field' : 'user', 'transform' : color_mapper},
                  size = {'field' : 'FixationDuration', 'transform' : size_mapper},
                  fill_alpha = 0.5,
                  legend_label = 'user')


gaze_plot.y_range.flipped = True

In [9]:
layout = row (
    gaze_plot, 
    column (select_map, select_user, slider_timestamp)
)

update()

curdoc().add_root(layout)

show(layout)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

